In [1]:
import os
import sys
from glob import glob

import cartopy.crs as ccrs
import cmocean
import dask
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

# Local Utils
sys.path.insert(0, "/glade/work/zespinosa/Projects/utils")
from utils import (
    plot_stationary_sp,
    xarray_monthly_to_time,
    xarray_time_to_monthly,
    detrend_data,
)

TODO (10/7/2022): 
1. Create repo for utils
2. Compare forecast
3. Compare extreme events
4. Sanity check with Turner paper
5. Add CC about sea ice thickness product
6. Create last few months of era5
7. Alias cm on git

In [3]:
def calc_sia_and_sie_cesm(siconc, grid):
    """
    Calculate sea ice area and sea ice extent from nsidc data
    
    Arguments:
    -----------
        siconc [Dataset, DataArray](..., latitude, longitude)

    Returns:
    --------
        si [Dataset](..., sia, sie)
    """

    sia = (siconc * grid).sum(["longitude", "latitude"], skipna=True, min_count=100) / 1e6
    sie = xr.where(siconc >= 0.15, grid, 0).sum(["longitude", "latitude"], skipna=True, min_count=100) / 1e6
    
    si = xr.Dataset(data_vars={"sia": sia, "sie": sie})
    si.attrs['processed'] = 'SIA and SIE computed by Zac Espinosa'
    
    return si
    
# Plot sea ice concentratio 
def spatial_cesm_plot(siconc, lon, lat, title):
    fig, ax = plot_stationary_sp()
    # Plot siconc filled contours
    img = ax.contourf(
        lon,
        lat, 
        siconc/100,
        transform=ccrs.PlateCarree(), 
        levels=np.arange(0, 1.1, .1),
        cmap=cmocean.cm.ice,
    )
    cbar2 = fig.colorbar(img, ax=ax)
    ax.set_title(title)
    fig.set_size_inches(5, 5)

Load CESM siconc data, calculate sia and sie in regions, and get detrended anomalies

In [4]:
# Load cesm-nudged siconc
cesm_nudged_siconc = xr.open_dataset("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm_nudged_siconc_198501-202412.nc")
cesm_areacello = xr.open_dataset("/glade/work/zespinosa/GRIDS/areacello_fx_CESM1-CAM5_historical_r0i0p0.nc").areacello
cesm_areacello = cesm_areacello.rename({"j": "longitude", "i": 'latitude'})

In [8]:
# Calculate sia and sie for each region
def calc_regions_cesm(siconc, grid, regional_bounds=None, sanity_check=True):
    if regional_bounds == None:
        regional_bounds = {
            "ross": (155, 215),
            "amundsen": (215, 255),
            "bellingshausen": (255, 295),
            "weddell": (295, 360),
            "south_indian": (0, 75),
            "south_west_pacific": (75, 155),
            "total": (0, 360),
        }
    
    siconc = siconc.sortby(siconc.TLON[0,:])
    grid = grid.sortby(siconc.TLON[0,:])
    
    si_regions = []
    for i, (reg, (lonMin, lonMax)) in enumerate(regional_bounds.items()):
        siconc_region = siconc.aice.where(((siconc.TLON >= lonMin) & (siconc.TLON < lonMax)), np.nan)        
        if sanity_check: spatial_cesm_plot(siconc_region.sel(year=2021, month=8), siconc.TLON, siconc.TLAT, reg)
        si_regions.append(calc_sia_and_sie_cesm(siconc_region, grid))
        
    si_regions = xr.concat(si_regions, dim="region")
    si_regions["region"] = list(regional_bounds.keys())
    return si_regions


siCESM = calc_regions_cesm(cesm_nudged_siconc, cesm_areacello, sanity_check=False)
siCESM.to_netcdf("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm_nudged_sia_sie_198501-202412.nc")

siaCESM_anoms = detrend_data(siCESM.sia, siCESM.year, "year", deg=1)
sieCESM_anoms = detrend_data(siCESM.sie, siCESM.year, "year", deg=1)
siaCESM_anoms.to_netcdf("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm_nudged_sia_anoms_198501-202412.nc")
sieCESM_anoms.to_netcdf("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm_nudged_sie_anoms_198501-202412.nc")

Load NSIDC sia and sie in regions and get detrended anomalies

In [11]:
siNSIDC_regions = xr.open_dataset("/glade/work/zespinosa/data/nsidc/processed/sia_sie_NSIDC_197901-202210.nc")
siNSIDC_regions = xarray_time_to_monthly(siNSIDC_regions)
siaNSIDC_anoms = detrend_data(siNSIDC_regions.sia, siNSIDC_regions.year, "year", deg=1)
sieNSIDC_anoms = detrend_data(siNSIDC_regions.sie, siNSIDC_regions.year, "year", deg=1)

Compare regional time series

In [ ]:
def regional_plots():
    fig, axs = plt.subplots(4,2, figsize=(15, 15))
    for cols in axs: 
        for ax in cols:
            ax.plot(siaNSIDC_anoms.sel(region="ross").mean(["month"]), label="cesm")
            ax.plot((siaCESM_anoms.sel(region="ross").mean(["month"]) / 1e8), label="nsidc") 
    
    fig.delaxes(axs[-1,-1])

regional_plots()